In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, GenerationConfig, DataCollatorWithPadding
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler
import torch
import evaluate
import numpy as np
import pandas as pd
from tqdm import tqdm

/home2/likhithasapu/miniconda3/envs/research/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
from huggingface_hub import login


data = load_dataset("kapilrk04/codemix-en_enhi",split="train",use_auth_token=True,cache_dir="codemix-dataset")
data = data.train_test_split(test_size=0.2,shuffle=True)
print(data)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home2/likhithasapu/.cache/huggingface/token
Login successful


/home2/likhithasapu/miniconda3/envs/research/lib/python3.11/site-packages/datasets/load.py:2069: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 1003028
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 250757
    })
})


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home2/likhithasapu/.cache/huggingface/token
Login successful


In [4]:
# Load tokenizers and models
translation_model_name = "likhithasapu/codemix-indicbart"
acceptability_model_name = "likhithasapu/indic-bert-regression-v1"
tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(translation_model_name)
acceptability_model = AutoModelForSequenceClassification.from_pretrained(acceptability_model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Preprocess function
def preprocess_function(examples):
    prompts = [f"Translate the English sentence to Hindi-English sentence: <s> {example['en']} </s>" for example in examples['translation']]
    responses = [f"<s> {example['en-hi']} </s>" for example in examples['translation']]
    
    model_inputs = tokenizer(prompts, truncation=True, padding=True, max_length=256)
    labels = tokenizer(responses, truncation=True, padding=True, max_length=256)["input_ids"]
    
    model_inputs["labels"] = labels
    return model_inputs

In [6]:

# Apply the preprocess function to the dataset with batching
batch_size = 64
processed_data = data.map(
    preprocess_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=data["train"].column_names
)


Map:   0%|          | 0/1003028 [00:00<?, ? examples/s]

Map: 100%|██████████| 250757/250757 [00:40<00:00, 6148.58 examples/s]


In [7]:
# Define PPO configuration
ppo_config = PPOConfig(
    model_name=translation_model_name,    
    learning_rate=1.41e-5,
    ppo_epochs=1,
    mini_batch_size=2,
    batch_size=8
)

In [8]:
# Define reward function
def compute_reward(predictions):
    inputs = tokenizer(predictions, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(acceptability_model.device) for k, v in inputs.items()}
    with torch.no_grad():
        scores = acceptability_model(**inputs).logits.squeeze().tolist()
        
    return scores

In [9]:
from torch.utils.data import DataLoader
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(processed_data["train"], batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(processed_data["test"], batch_size=8, collate_fn=data_collator)

In [10]:
# Initialize PPO model
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(translation_model)
ref_model = create_reference_model(ppo_model)
ppo_trainer = PPOTrainer(config=ppo_config, model=ppo_model, ref_model=ref_model, tokenizer=tokenizer, dataset=processed_data["train"], data_collator=data_collator)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
# Training loop
max_ppo_steps = 10000
# Set up length sampler and generation configurations
output_min_length = 10
output_max_length = 100


# Training loop
for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if step >= max_ppo_steps:
        break   

    prompt_tensors = [tensor for tensor in batch["input_ids"]]
    attention_mask = [tensor for tensor in batch["attention_mask"]]

    # Get response from the model
    summary_tensors = []

    for prompt_tensor,attention_tensor in zip(prompt_tensors,attention_mask):        
        summary = ppo_model.generate(input_ids = prompt_tensor.unsqueeze(0).to("cuda"), attention_mask = attention_tensor.unsqueeze(0).to("cuda"), max_new_tokens=output_max_length, num_beams=5, no_repeat_ngram_size=3, do_sample=True, num_return_sequences=1)
        summary_tensors.append(summary.squeeze()[-output_max_length:])
        
    batch["response"] = [tokenizer.decode(r.squeeze(), skip_special_tokens=True) for r in summary_tensors]

    # Compute reward outputs.
    query_response_pairs = [tokenizer.decode(q, skip_special_tokens=True) + r for q, r in zip(batch["input_ids"], batch["response"])]    
    rewards = [torch.tensor(reward) for reward in compute_reward(query_response_pairs)]
    print(rewards)
    # Run PPO step.
    
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]

[tensor(3.1194), tensor(3.1821), tensor(3.1052), tensor(3.1015), tensor(3.0763), tensor(3.1072), tensor(3.0079), tensor(3.0481)]


/home2/likhithasapu/miniconda3/envs/research/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -13.10 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
1it [00:05,  5.14s/it]

[tensor(3.4473), tensor(3.1617), tensor(3.0443), tensor(3.0003), tensor(3.1177), tensor(3.1408), tensor(3.1187), tensor(3.0363)]


/home2/likhithasapu/miniconda3/envs/research/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -19.06 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
2it [00:09,  4.95s/it]

...

[tensor(3.2065), tensor(3.2590), tensor(3.0876), tensor(3.3323), tensor(3.3033), tensor(3.1366), tensor(3.3350), tensor(3.3393)]


/home2/likhithasapu/miniconda3/envs/research/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -92.22 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
164it [23:49,  9.53s/it]

[tensor(3.1279), tensor(3.4538), tensor(3.0480), tensor(3.1522), tensor(3.3322), tensor(3.5140), tensor(3.2733), tensor(3.5368)]


/home2/likhithasapu/miniconda3/envs/research/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -96.17 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
165it [24:01,  8.74s/it]


KeyboardInterrupt: 

In [ ]:
# Save the model
tokenizer.push_to_hub("likhithasapu/codemix-indicbart-ppo-1000")
ppo_model.push_to_hub("likhithasapu/codemix-indicbart-ppo-1000")